# Seattle Building Energy Forecasting

URL: https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking

## Notebook n°5 - Modelling

Objective: Create dummy variables and try different models

---

## Import librairies

In [ ]:
import time
import warnings
import re
import pickle
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
import xgboost as xgb

## Settings

In [194]:
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 200

target1 = "SiteEnergyUse(kBtu)"
target2 = "TotalGHGEmissions"

TARGET = target2

## Import data

In [195]:
with open("data/part4.pkl", "rb") as f:
    my_unpickler = pickle.Unpickler(f)
    data = my_unpickler.load()
    
with open("data/part4-data-with-outliers.pkl", "rb") as f:
    my_unpickler = pickle.Unpickler(f)
    data_with_outliers = my_unpickler.load()

## Functions

### Dummify variables

In [196]:
def onehot(data, feature):
    
    # Get dummy variables
    temp_df = pd.get_dummies(data[feature])
    
    # Add prefix to prevent duplicated feature names
    temp_df = temp_df.add_prefix(feature + "_")
    
    # Concatenante the new features with the main dataframes
    data = pd.concat([data, temp_df], axis=1)
    
    # Drop the original feature
    data.drop(feature, axis=1, inplace=True)
    
    # Return the new dataframe
    return data

### Prepare datasets

In [208]:
def prepare_data(data, energystarscore="fill"):
    
    # Copy original data
    data_copy = data.copy()
    
    # Building Type
    data_copy = onehot(data_copy, "BuildingType")
    
    # CouncilDistrictCode
    data_copy = onehot(data_copy, "CouncilDistrictCode")
    
    # Neighborhood
    data_copy = onehot(data_copy, "Neighborhood")
    
    # LargestPropertyUseType
    data_copy = onehot(data_copy, "LargestPropertyUseType")
    
    # PrimaryPropertyType
    data_copy = onehot(data_copy, "PrimaryPropertyType")
    
    # DataYear
    data_copy.drop("DataYear", axis=1, inplace=True)
    
    # Address
    data_copy.drop("Address", axis=1, inplace=True)
    
    # address_type
    data_copy.drop("address_type", axis=1, inplace=True)
    
    # lat_long_range
    data_copy.drop("lat_long_range", axis=1, inplace=True)
    
    # ZipCode
    data_copy.drop("ZipCode", axis=1, inplace=True)
    
    # default_data
    data_copy.drop("default_data", axis=1, inplace=True)
    
    # ENERGYSTARScore
    if (energystarscore == "fill"):
        data_copy["ENERGYSTARScore"].fillna(data_copy["ENERGYSTARScore"].dropna().mean(), inplace=True)
    elif (energystarscore == "drop"):
        mask = data_copy["ENERGYSTARScore"].isna()
        data_copy = data_copy[~mask]

    return data_copy.reset_index(drop=True)

### Random Forest Model

In [298]:
def my_rf(data, target, param_grid, test_size, target_log=False):

    X = data.drop(target, axis=1)
    y = data[target]
    
    if target_log:
        y = np.log(data[target])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    score = 'neg_mean_squared_error'

    # Cross validation classifier to get best parameters from the grid search
    clf_rf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring=score)
    clf_rf.fit(X_train_scaled, y_train)
    params = clf_rf.best_params_
        
    # Train prediction
    y_train_pred = clf_rf.predict(X_train_scaled)
    
    # Test prediction
    y_test_pred = clf_rf.predict(X_test_scaled)
    
    ############## Scores ##############
    
    # Get back to exponantial if we are in log for the target
    if target_log:
        y_train_pred = np.exp(y_train_pred)
        y_train = np.exp(y_train)
        y_test_pred = np.exp(y_test_pred)
        y_test = np.exp(y_test)
    
    # Compute R2 and RMSE for both training set and test set
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Push the score in a dict
    scores = {
        'train_rmse': train_rmse,
        'train_r2': train_r2,
        'test_rmse': test_rmse,
        'test_r2': test_r2
    }
    
    # Return the classifier, y test and scores
    return clf_rf, y_test, y_test_pred, scores, X_test

### XGboost Model

In [302]:
def my_xgb(data, params, target, test_size=0.2):
    """This function performs a cross-validation test on the given data. For now, it's designed to run with the XGBoost algorithm.
    
    First, it splits the features and the target.
    Then, it performs a train / test split if test_size != 0.
    Then, it performs a K-Folds cross-validation with sklearn.
    Finally, for each parameters combination it gives :
    - The cross-validation score
    - The training score
    - The test score (if test_size != 0)
    
    The scores are R2 and RMSE, so the function is designed only for regression so far.
    
    Args:
    
        data (Pandas Dataframe): the original dataset
        params (dict): the grid-search parameters. Each value has to be a list
        target (str): The name of the target as written in the dataset column
        test_size (float): The proportion of the test set ([0, 1[)
        
    Returns:
        Dict: All the results for each parameters combination plus the corresponding model trained on the entire training set.
    
    """
    
    #########################################################################
    # Prepare data : X / y split, then Train / test split, then scale
    #########################################################################
    
    X = data.drop(target, axis=1)
    y = data[target]
    
    if test_size != 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
    else:
        X_train = X
        y_train = y

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    #########################################################################
    # Prepare params for gridsearch. Compute all combinations
    #########################################################################
    
    params_keys = list(params.keys())
    params_product = list(itertools.product(*list(params.values())))
    params_flatten = {params_keys[i]: [item[i] for item in params_product] for i in range(len(params_keys))}
    
    #########################################################################
    # Prepare folds for cross-validation
    #########################################################################
    
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    
    #########################################################################
    # Prepare the results
    #########################################################################
    
    results = {
        'params': [],
        'cv_r2_score': [],
        'cv_rmse_score': [],
        'train_r2_score': [],
        'train_rmse_score': [],
    }
    
    if test_size != 0:
        results['test_r2_score'] = []
        results['test_rmse_score'] = []
        
    results['model'] = []
    
    #########################################################################
    # Parse all params combinations:
    #########################################################################
    
    for i in range(len(params_flatten[list(params_flatten.keys())[0]])):
        
        # Construct the object "temp_params"
        temp_params = {key: params_flatten[key][i] for key in params_flatten.keys()}
        
        # Cross Validation - Iterate over folds
        temp_scores = {
            'r2':[],
            'rmse':[]
        }
        for train_index, test_index in kf.split(X_train_scaled):

            # Train / Test split
            
            # X is a numpy array, so we just need to use indexes returned by the split
            X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
            
            # y is an indexed Pandas series, so we need to use .iloc
            y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

            # Train the model
            dtrain = xgb.DMatrix(X_train_cv, y_train_cv)
            dtest = xgb.DMatrix(X_test_cv)
            bst = xgb.train(temp_params, dtrain)
            y_test_cv_pred = bst.predict(dtest)

            # Save the score
            temp_scores['r2'].append(r2_score(y_test_cv, y_test_cv_pred))
            temp_scores['rmse'].append(np.sqrt(mean_squared_error(y_test_cv, y_test_cv_pred)))
        
        # Train the model on the entire train set
        dtrain = xgb.DMatrix(X_train_scaled, y_train)
        bst = xgb.train(temp_params, dtrain)
        
        # "Prediction" on train set
        y_train_pred = bst.predict(dtrain)
        train_r2_score = r2_score(y_train, y_train_pred)
        train_rmse_score = np.sqrt(mean_squared_error(y_train, y_train_pred))
        
        # Predict on the test set
        if test_size != 0:
            dtest = xgb.DMatrix(X_test_scaled)
            y_test_pred = bst.predict(dtest)
            test_r2_score = r2_score(y_test, y_test_pred)
            test_rmse_score = np.sqrt(mean_squared_error(y_test, y_test_pred))
        
        # Save the final scores for the given params
        results['params'].append(temp_params)
        results['cv_r2_score'].append(np.mean(temp_scores['r2']))
        results['cv_rmse_score'].append(np.mean(temp_scores['rmse']))
        results['train_r2_score'].append(train_r2_score)
        results['train_rmse_score'].append(train_rmse_score)
        results['model'].append(bst)
        
        if test_size != 0:
            results['test_r2_score'].append(test_r2_score)
            results['test_rmse_score'].append(test_rmse_score)
        
    return results

### Errors Analysis

In [199]:
def error_analysis(X, y_true, y_pred):
    
    errors = y_pred - y_true
    
    sns.scatterplot(y_true, y_pred)
    plt.title("y_pred as a function of y_true", fontweight="bold")
    plt.xlabel("y_true", fontweight="bold")
    plt.ylabel("y_pred", fontweight="bold")
    plt.show()
    
    sns.distplot(errors, kde=False)
    plt.title("Errors distribution", fontweight="bold")
    plt.xlabel("Error value", fontweight="bold")
    plt.ylabel("Total errors", fontweight="bold")
    plt.show()
    
    sns.scatterplot(y_true, errors)
    plt.title("Error value as a function of y_true", fontweight="bold")
    plt.xlabel("y_true", fontweight="bold")
    plt.ylabel("Error value", fontweight="bold")
    plt.show()

## Prepare datasets

In [209]:
data_prep = prepare_data(data)
data_with_outliers_prep = prepare_data(data_with_outliers)

### Site Energy Use - Random Forest

In [299]:
param_grid = {'n_estimators': [600],
              'max_features': ['auto'],
              'n_jobs': [-1],
              'random_state': [42],
              'max_depth': [None],
              'min_samples_leaf': [1],
              'min_samples_split': [2],
              }

classifier1, y_test1, y_pred1, scores1, X_test1 = my_rf(data_prep.drop([target2, "ENERGYSTARScore"], axis=1), target1, param_grid, test_size=0.20)
display(classifier1.best_params_)
display(scores1)

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'n_jobs': -1,
 'random_state': 42}

{'train_rmse': 526155.9997101225,
 'train_r2': 0.9801344581186169,
 'test_rmse': 1232320.5184747213,
 'test_r2': 0.8801388862977408}

### Site Energy Use - XGboost

In [306]:
params = {'max_depth': [10, 20, 30, 50],
          'eta': [0.1, 0.3, 0.5],
          'min_child_weight': [1, 5, 10],
          'subsample': [0.6, 0.8, 1.0],
          'colsample_bytree': [0.6, 0.8, 1.0],
          'objective': ['reg:squarederror'],
          'nthread': [4]}

classifier2 = my_xgb(data_prep.drop([target2, "ENERGYSTARScore"], axis=1), params, target1)

In [307]:
display(pd.DataFrame(classifier2).sort_values(by="test_rmse_score").head(5))

,params,cv_r2_score,cv_rmse_score,train_r2_score,train_rmse_score,test_r2_score,test_rmse_score,model
301,"{'max_depth': 50, 'eta': 0.5, 'min_child_weigh...",0.756758,1.837453e+06,0.997798,175184.242238,0.901592,1.116608e+06,<xgboost.core.Booster object at 0x1354765f8>
220,"{'max_depth': 30, 'eta': 0.5, 'min_child_weigh...",0.761256,1.820608e+06,0.997734,177695.140451,0.900955,1.120212e+06,<xgboost.core.Booster object at 0x12f8b2f60>
142,"{'max_depth': 20, 'eta': 0.5, 'min_child_weigh...",0.795182,1.686408e+06,0.998782,130261.711119,0.900523,1.122655e+06,<xgboost.core.Booster object at 0x1282dd828>
274,"{'max_depth': 50, 'eta': 0.3, 'min_child_weigh...",0.783797,1.734048e+06,0.982691,491139.651906,0.900371,1.123512e+06,<xgboost.core.Booster object at 0x12f8bda58>
193,"{'max_depth': 30, 'eta': 0.3, 'min_child_weigh...",0.782434,1.740074e+06,0.982333,496188.751357,0.900339,1.123691e+06,<xgboost.core.Booster object at 0x12ab90ba8>


### Site Energy Use - XGBoost with outliers

In [ ]:
params = {'max_depth': [10, 20, 30, 50],
          'eta': [0.1, 0.3, 0.5],
          'min_child_weight': [1, 5, 10],
          'subsample': [0.6, 0.8, 1.0],
          'colsample_bytree': [0.6, 0.8, 1.0],
          'objective': ['reg:squarederror'],
          'nthread': [4]}

classifier3 = my_xgb(data_with_outliers_prep.drop([target2, "ENERGYSTARScore"], axis=1), params, target1)

In [ ]:
display(pd.DataFrame(classifier3).sort_values(by="test_rmse_score").head(5))

### TotalGHGEmissions - Random Forest

In [ ]:
param_grid = {'n_estimators': [600],
              'max_features': ['auto'],
              'n_jobs': [-1],
              'random_state': [42],
              'max_depth': [None],
              'min_samples_leaf': [1],
              'min_samples_split': [2],
              }

classifier4, y_test4, y_pred4, scores4, X_test4 = my_rf(data_prep.drop([target1, "ENERGYSTARScore"], axis=1), target2, param_grid, test_size=0.20)
display(classifier4.best_params_)
display(scores4)

### TotalGHGEmissions - XGBoost

In [ ]:
params = {'max_depth': [10, 20, 30, 50],
          'eta': [0.1, 0.3, 0.5],
          'min_child_weight': [1, 5, 10],
          'subsample': [0.6, 0.8, 1.0],
          'colsample_bytree': [0.6, 0.8, 1.0],
          'objective': ['reg:squarederror'],
          'nthread': [4]}

classifier5 = my_xgb(data_prep.drop([target1, "ENERGYSTARScore"], axis=1), params, target2)

In [ ]:
display(pd.DataFrame(classifier5).sort_values(by="test_rmse_score").head(5))

### TotalGHGEmissions - XGBoost with ENERGYStarScore

In [ ]:
params = {'max_depth': [10, 20, 30, 50],
          'eta': [0.1, 0.3, 0.5],
          'min_child_weight': [1, 5, 10],
          'subsample': [0.6, 0.8, 1.0],
          'colsample_bytree': [0.6, 0.8, 1.0],
          'objective': ['reg:squarederror'],
          'nthread': [4]}

classifier2 = my_xgb(data_prep.drop([target1], axis=1), params, target2)

In [ ]:
display(pd.DataFrame(classifier2).sort_values(by="test_rmse_score").head(5))